# Libraries

In [25]:
import numpy as np
import scipy
from scipy import signal
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import math

# root = str(Path().resolve().parent)
custom_pkg = str(Path().resolve().parent.parent / "python")
if custom_pkg not in sys.path:
    sys.path.append(custom_pkg)
print(*sys.path, sep="\n")

# %matplotlib ipympl
%matplotlib inline

/usr/lib/python312.zip
/usr/lib/python3.12
/usr/lib/python3.12/lib-dynload

/home/ryanwtsai/repos/rfdsp_sim/python/venv/lib/python3.12/site-packages
/home/ryanwtsai/repos/rfdsp_sim/python


In [26]:
from importlib import reload
from rfdsppy import calc, ofdm, rf_analog, rf_tx_fw, tonegen, digital_filter_design, digital_hw_algo as dighw

reload(calc)
reload(ofdm)
reload(rf_analog)
reload(rf_tx_fw)
reload(tonegen)
reload(digital_filter_design)
reload(dighw)

<module 'rfdsppy.digital_hw_algo' from '/home/ryanwtsai/repos/rfdsp_sim/python/rfdsppy/digital_hw_algo.py'>

# Polyphase decomposition (DTSP)

In [4]:
# Prototype filter

M = 3
fp = 0.2
b = signal.firls(15, [0, fp, 2/M-fp, 1], [1, 1, 0, 0], fs=2)
b = b/b.sum()
b = np.round(b*(2**15-1))

print(b.astype("int"))

[  329   151  -816 -1597  -258  3956  8988 11262  8988  3956  -258 -1597
  -816   151   329]


In [5]:
# Polyphase components

e = []
for i in range(M):
    e.append(b[i::M])

print(*e, sep="\n")

[  329. -1597.  8988.  3956.  -816.]
[  151.  -258. 11262.  -258.   151.]
[ -816.  3956.  8988. -1597.   329.]


In [6]:
# Reconstruct prototype filter

e_up = []
for i in range(M):
    e_up.append(np.roll(np.concatenate((np.zeros(i), dighw.upsample(e[i], M))), i)[i:])

# print(*e_up, sep="\n")
print(np.array(e_up))

[[  329.     0.     0. -1597.     0.     0.  8988.     0.     0.  3956.
      0.     0.  -816.     0.     0.]
 [    0.   151.     0.     0.  -258.     0.     0. 11262.     0.     0.
   -258.     0.     0.   151.     0.]
 [    0.     0.  -816.     0.     0.  3956.     0.     0.  8988.     0.
      0. -1597.     0.     0.   329.]]


In [7]:
# Polyphase components

e = np.zeros((M, math.ceil(b.size/M)))
for i in range(M):
    e[i, :] = b[i::M]

print(e)

[[  329. -1597.  8988.  3956.  -816.]
 [  151.  -258. 11262.  -258.   151.]
 [ -816.  3956.  8988. -1597.   329.]]


In [8]:
# Upsample and shift polyphase components

e_up = np.zeros((M, e.shape[1]*M))
for i in range(M):
    e_up[i] = np.roll(np.concatenate((np.zeros(i), dighw.upsample(e[i], M))), i)[i:]

print(e_up)

# Reconstruct prototype filter

print(e_up.sum(axis=0))
print(e_up.sum(axis=0) - b)

[[  329.     0.     0. -1597.     0.     0.  8988.     0.     0.  3956.
      0.     0.  -816.     0.     0.]
 [    0.   151.     0.     0.  -258.     0.     0. 11262.     0.     0.
   -258.     0.     0.   151.     0.]
 [    0.     0.  -816.     0.     0.  3956.     0.     0.  8988.     0.
      0. -1597.     0.     0.   329.]]
[  329.   151.  -816. -1597.  -258.  3956.  8988. 11262.  8988.  3956.
  -258. -1597.  -816.   151.   329.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
b

array([  329.,   151.,  -816., -1597.,  -258.,  3956.,  8988., 11262.,
        8988.,  3956.,  -258., -1597.,  -816.,   151.,   329.])

In [10]:
e

array([[  329., -1597.,  8988.,  3956.,  -816.],
       [  151.,  -258., 11262.,  -258.,   151.],
       [ -816.,  3956.,  8988., -1597.,   329.]])

In [11]:
(e[0]+e[2])/2

array([-243.5, 1179.5, 8988. , 1179.5, -243.5])

In [12]:
(e[0]-e[2])/2

array([  572.5, -2776.5,     0. ,  2776.5,  -572.5])

# Polyphase decomposition (restoring coefficient symmetry)

In [17]:
L = 31
b = np.arange(L)
b[math.ceil(L/2):] = b[math.floor(L/2)-1::-1]
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 14,
       13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [23]:
print(dighw.polyphase_decomposition(b, 2))
print("")
print(dighw.polyphase_decomposition(b, 3))
print("")
print(dighw.polyphase_decomposition(b, 4))
print("")
print(dighw.polyphase_decomposition(b, 5))

[[ 0.  2.  4.  6.  8. 10. 12. 14. 14. 12. 10.  8.  6.  4.  2.  0.]
 [ 1.  3.  5.  7.  9. 11. 13. 15. 13. 11.  9.  7.  5.  3.  1.  0.]]

[[ 0.  3.  6.  9. 12. 15. 12.  9.  6.  3.  0.]
 [ 1.  4.  7. 10. 13. 14. 11.  8.  5.  2.  0.]
 [ 2.  5.  8. 11. 14. 13. 10.  7.  4.  1.  0.]]

[[ 0.  4.  8. 12. 14. 10.  6.  2.]
 [ 1.  5.  9. 13. 13.  9.  5.  1.]
 [ 2.  6. 10. 14. 12.  8.  4.  0.]
 [ 3.  7. 11. 15. 11.  7.  3.  0.]]

[[ 0.  5. 10. 15. 10.  5.  0.]
 [ 1.  6. 11. 14.  9.  4.  0.]
 [ 2.  7. 12. 13.  8.  3.  0.]
 [ 3.  8. 13. 12.  7.  2.  0.]
 [ 4.  9. 14. 11.  6.  1.  0.]]


In [24]:
print(dighw.polyphase_decomposition(b, 2, mode="symmetric"))
print("")
print(dighw.polyphase_decomposition(b, 3, mode="symmetric"))
print("")
print(dighw.polyphase_decomposition(b, 4, mode="symmetric"))
print("")
print(dighw.polyphase_decomposition(b, 5, mode="symmetric"))

[[ 0.  1.  3.  5.  7.  9. 11. 13. 15. 13. 11.  9.  7.  5.  3.  1.  0.]
 [ 0.  2.  4.  6.  8. 10. 12. 14. 14. 12. 10.  8.  6.  4.  2.  0.  0.]]

[[ 0.   3.   6.   9.  12.  15.  12.   9.   6.   3.   0. ]
 [ 1.5  4.5  7.5 10.5 13.5 13.5 10.5  7.5  4.5  1.5  0. ]
 [-0.5 -0.5 -0.5 -0.5 -0.5  0.5  0.5  0.5  0.5  0.5  0. ]]

[[ 0.  3.  7. 11. 15. 11.  7.  3.  0.]
 [ 1.  5.  9. 13. 13.  9.  5.  1.  0.]
 [ 1.  5.  9. 13. 13.  9.  5.  1.  0.]
 [-1. -1. -1. -1.  1.  1.  1.  1.  0.]]

[[ 0.   5.  10.  15.  10.   5.   0. ]
 [ 2.5  7.5 12.5 12.5  7.5  2.5  0. ]
 [ 2.5  7.5 12.5 12.5  7.5  2.5  0. ]
 [-0.5 -0.5 -0.5  0.5  0.5  0.5  0. ]
 [-1.5 -1.5 -1.5  1.5  1.5  1.5  0. ]]
